Jupter com a utilização de PCA antes da RNN

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

Ler os dados dos csv com os dados simulados contidos nas pastas

In [1]:
def ler_csv_e_atribuir_classes_com_pca(diretorio, classe, n_components=10):
    dados = []
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(diretorio, arquivo)
            df = pd.read_csv(caminho_arquivo, sep=';')
            features = df.iloc[:, :-1]  # Extrair features para PCA
            features_normalizadas = (features - features.mean()) / features.std()  # Normalização
            pca = PCA(n_components=n_components)  # PCA
            features_reduzidas = pca.fit_transform(features_normalizadas)  # Aplicar PCA
            df_reduzido = pd.DataFrame(data=features_reduzidas, columns=[f'componente_{i}' for i in range(1, n_components + 1)])
            df_reduzido['Classe'] = classe
            dados.append(df_reduzido)
    return pd.concat(dados, ignore_index=True)

diretorio_antes = './bkp_datasets_model/antes'
diretorio_depois = './bkp_datasets_model/depois'

dados_antes = ler_csv_e_atribuir_classes_com_pca(diretorio_antes, classe=0)

dados_depois = ler_csv_e_atribuir_classes_com_pca(diretorio_depois, classe=1)

dados_combinados = pd.concat([dados_antes, dados_depois], ignore_index=True)

In [18]:
dados_combinados.shape

(2310000, 11)

In [19]:
import numpy as np

def split_into_sequences(data, sequence_length=10000):
    sequences = []
    labels = []
    for i in range(0, len(data), sequence_length):
        sequence = data.iloc[i:i+sequence_length, :-1]  # Excluir a última coluna (classe)
        label = data.iloc[i+sequence_length-1, -1]  # Última classe da sequência
        sequences.append(sequence.values)
        labels.append(label)
    return np.array(sequences), np.array(labels)

X_sequences, y_sequences = split_into_sequences(dados_combinados)

print("Dimensões das sequências de características:", X_sequences.shape)
print("Dimensões das sequências de rótulos:", y_sequences.shape)


Dimensões das sequências de características: (231, 10000, 10)
Dimensões das sequências de rótulos: (231,)


Separar e verificar as dimensões dos dados de treinamento e teste

In [20]:
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# Verificar as dimensões dos dados de treinamento e teste
print("Dimensões dos dados de treinamento de características:", X_train.shape)
print("Dimensões dos dados de teste de características:", X_test.shape)
print("Dimensões dos dados de treinamento de rótulos:", y_train.shape)
print("Dimensões dos dados de teste de rótulos:", y_test.shape)

Dimensões dos dados de treinamento de características: (184, 10000, 10)
Dimensões dos dados de teste de características: (47, 10000, 10)
Dimensões dos dados de treinamento de rótulos: (184,)
Dimensões dos dados de teste de rótulos: (47,)


Realização do treinamento

In [14]:
checkpoint_path = "model_checkpoint_pca_ger.h5"

checkpoint_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

model = Sequential([
    LSTM(10, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[checkpoint_callback])

Epoch 1/10
22/22 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.5185
Epoch 1: val_accuracy improved from -inf to 0.46023, saving model to model_checkpoint_pca_ger.h5
22/22 [==============================] - 110s 5s/step - loss: 0.6930 - accuracy: 0.5185 - val_loss: 0.6989 - val_accuracy: 0.4602
Epoch 2/10
22/22 [==============================] - ETA: 0s - loss: 0.6854 - accuracy: 0.5582
Epoch 2: val_accuracy improved from 0.46023 to 0.52841, saving model to model_checkpoint_pca_ger.h5
22/22 [==============================] - 110s 5s/step - loss: 0.6854 - accuracy: 0.5582 - val_loss: 0.6947 - val_accuracy: 0.5284
Epoch 3/10
22/22 [==============================] - ETA: 0s - loss: 0.6754 - accuracy: 0.5980
Epoch 3: val_accuracy did not improve from 0.52841
22/22 [==============================] - 113s 5s/step - loss: 0.6754 - accuracy: 0.5980 - val_loss: 0.6981 - val_accuracy: 0.5114
Epoch 4/10
22/22 [==============================] - ETA: 0s - loss: 0.6605 - acc

In [21]:
import h5py

with h5py.File(checkpoint_path, 'r') as f:
    best_model = tf.keras.models.load_model(f)


In [22]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Acurácia do melhor modelo: {accuracy}')

2/2 [==============================] - 2s 448ms/step - loss: 0.7464 - accuracy: 0.5532
Acurácia do melhor modelo: 0.5531914830207825


In [24]:
loss, accuracy = best_model.evaluate(X_train, y_train)
print(f'Acurácia do melhor modelo: {accuracy}')

6/6 [==============================] - 4s 596ms/step - loss: 0.7288 - accuracy: 0.5489
Acurácia do melhor modelo: 0.5489130616188049
